In [1]:
import dask
from fake_useragent import UserAgent
from SPARQLWrapper import SPARQLWrapper, JSON, XML
import pandas as pd
import json
import time
import bookquery
import os
import io
import json
from tqdm import tqdm
from os.path import join,isfile
from os import listdir
from collections import Counter
import dask
import openpyxl

def counter_for_files(path):
    
    """Подсчет файлов в директории"""
        
    onlyfiles = [join(path, f) for f in listdir(path) if isfile(join(path, f))]
    return onlyfiles

In [2]:
# with open(r'D:\Univer\Сбербанк\Work\Geography\sber\wikidata\artificial_classes\data\geography\v8\artificial_w_singletons_v8.json') as f:
#     d = json.load(f)

# with open(r'D:\Univer\Сбербанк\Work\Geography\sber\wikidata\artificial_classes\data\geography\v8\singletons_v8.json') as f:
#     d2 = json.load(f)
    
# with open(r'D:\Univer\Сбербанк\Work\Geography\sber\wikidata\artificial_classes\data\geography\v8\artificial_v8.json') as f:
#     d3 = json.load(f)
    
# with open(r'D:\Univer\Сбербанк\Work\Geography\sber\wikidata\artificial_classes\data\geography\v8\singletons_inv_v8.json') as f:
#     d4 = json.load(f)
    
    
    
with open(r'C:\Users\Алексей Кушнер\sber_basic\BOCK2\wikidata\artificial_classes\data\geography\v8\artificial_w_singletons_v8.json') as f:
    d = json.load(f)

with open(r'C:\Users\Алексей Кушнер\sber_basic\BOCK2\wikidata\artificial_classes\data\geography\v8\singletons_v8.json') as f:
    d2 = json.load(f)
    
with open(r'C:\Users\Алексей Кушнер\sber_basic\BOCK2\wikidata\artificial_classes\data\geography\v8\artificial_v8.json') as f:
    d3 = json.load(f)
    
with open(r'C:\Users\Алексей Кушнер\sber_basic\BOCK2\wikidata\artificial_classes\data\geography\v8\singletons_inv_v8.json') as f:
    d4 = json.load(f)
        

In [3]:
ua = UserAgent()
bad_requests = []

def execute_query(qquery, entity_id):
    
    """Базовая функция для связи с сервером"""
    
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql", agent = ua.random)
    sparql.setQuery(qquery.format(entity_id = entity_id))
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    return results['results']['bindings']

def entities_sparqul(entity_id, f = 0):
    
    """Получение ответа с сервера и формирование датафреймов"""
    
    res = []
    if f > 5:
        global bad_requests
        bad_requests.append(entity_id)
        print('Bad request ', entity_id)
        return None
    try:
        results= execute_query(bookquery.subclass_with_parents, entity_id)
        for result in results:
            entities = {}
            entities['subclass'] = result['show']['value'].split('/')[-1]
            entities['subclasslabel'] = result['showLabel']['value']
            entities['classid'] = result['parent']['value'].split('/')[-1]
            entities['classlabel'] = result['parentLabel']['value']
            res.append(entities)
        return pd.DataFrame(data = res, columns = ['classid', 'classlabel' ,'subclass', 'subclasslabel'])
    except Exception as e:
        print(e)
        time.sleep(5)
        print('Try again ', entity_id, f)
        return entities_sparqul(entity_id, f + 1)
        
# def getting_subclass(subclasses , name):
#     main_df = pd.DataFrame()
#     for item in subclasses:
#         df = entities_sparqul("Q4022")
#         df = df[df['classid'].isin(list(df.subclass))]
#         df =  df[lambda x : x['subclasslabel'].str[0]!='Q']
#         new_df = df.groupby(['subclass' , 'subclasslabel']).apply(lambda x: ','.join(list(map(lambda x: '_'.join(x) , zip(x.classid,x.classlabel))))).reset_index()
#         new_df.columns = ['subclass' , 'subclasslabel' , 'class']
#         new_df['signifance'] = ''
#         main_df = pd.concat([main_df , new_df])
#     main_df.to_excel(f'excel_with_subclasses/{name}.xlsx' , index = False)

def creating_pool(v):
    
    """Формирование списка всех QID"""

    for i in v:
        if i[0] == "Q":
            all_id.append(i)
        else:
            creating_pool(d3[i])

    all_id = []
    for k,v in d3.items():
        creating_pool(v)
    all_id = list(set(all_id))

def new_sheets(sheetname , ID , filename):
    
    """Функция формирования листов excel"""
    
#     print(sheetname , filename)
    if len(sheetname)>30:
        sheetname = sheetname[:30]
    df = entities_sparqul(ID)
    df = df[df['classid'].isin(list(df.subclass))] # Очищаем от классов без родителей
    df =  df[lambda x : x['subclasslabel'].str[0]!='Q']# Убираем все Q
    local_lst = []
    for k,v in df.iterrows():# В цикле чистим все повторяющиеся классы и их подклассы
        if (((v.classid in all_id) or (v.classid in local_lst)) and (v.classid != ID)) or (v.subclass in all_id):
            local_lst.append(v.subclass)
            df.drop(k,inplace = True)
    if len(df) == 0:
        return
    new_df = df.groupby(['subclass' , 'subclasslabel']).apply(
        lambda x: ','.join(list(map(lambda x: '_'.join(x) , zip(x.classid,x.classlabel))))).reset_index()# Формируем столбец классов
    new_df.columns = ['subclass' , 'subclasslabel' , 'class']
    if sheetname == filename or f"{filename}" not in list(map(lambda x: x.split("\\")[1].split('.')[0] , counter_for_files(r"excel_with_subclasses/basic"))):
        new_df.to_excel(f'excel_with_subclasses/basic/{filename}.xlsx', sheet_name = sheetname ,encoding='utf-8-sig' , index = False)
    else:
        with pd.ExcelWriter(f'excel_with_subclasses/basic/{filename}.xlsx' ,  engine="openpyxl" , mode  = 'a') as writer:
            new_df.to_excel(writer, sheet_name = sheetname , encoding='utf-8-sig' , index = False)
        
def new_file(name , dic):
    
    """Функция формирования файлов"""
    
    if f'{name}' not in list(map(lambda x: x.split("\\")[1].split('.')[0] , counter_for_files(r"excel_with_subclasses/basic"))):
        for k,v in dic.items():
            if k == name:
                new_sheets(k,v , name)
                del(dic[k])
                break
        for k,v in dic.items():
            new_sheets(k,v , name)
            
def subclass_json(k,v):
    
    """Формирование словариков для создания файлов"""
    
    new_d = {}
    for item in v:
        if item[0] == 'Q':
            new_d[k.split(':')[1]] = item
        elif item in d2.keys():
            new_d[item.split(':')[1]] = d2[item][0]
        else:
            subclass_json(item , d[item])
    return new_d

# all_id_new = {}
# for k , v in d3.items():
#     for i in v:
#         if i[0] == "Q":
#             if i in  all_id_new:
#                 all_id_new[i].append(k.split(':')[1])
#             else:
#                 all_id_new[i] = [k.split(':')[1]]
                

In [4]:
#СОздаем словарики для всех классов и синглтонов
arr = []
for k,v in d2.items():
    arr.append((subclass_json(k,v) , k.split(":")[1]))
for k,v in d.items():
    arr.append((subclass_json(k,v) , k.split(":")[1]))
    

In [5]:
all_id_new = {}
for i in arr:#Создание структуры для удаления дубликатов
    for k,v in i[0].items():
        if k in all_id_new:
            all_id_new[k].append((i[1] , len(i[0])))
        else:
            all_id_new[k] = [(i[1] , len(i[0]))]
                
duplicate = {}
for k,v in all_id_new.items():
    if len(v)>1:
        duplicate[k] = sorted(
            v , key = lambda x: x[1])# Тут сортируем по количеству подклассов класса, чтобы в первую очередь удалять синглтоны

In [6]:
#Очистка от дубликатов
for k,v in duplicate.items():
    i=0
    while (i+1<len(v)):
        for item in range(len(arr)):
            if arr[item][1] == v[i][0]:
                del arr[item][0][k]
                if len(arr[item][0]) == 0:
                    del arr[item]
                i +=1
                break

In [12]:
d4

{'Q944588': 'dr:slope',
 'Q2026833': 'dr:garden_square',
 'Q34763': 'dr:peninsula',
 'Q484215': 'dr:federal_district_of_russia',
 'Q88480': 'dr:moat',
 'Q1701967': 'dr:coastal_line',
 'Q2042028': 'dr:ravine',
 'Q1281105': 'dr:piste',
 'Q501094': 'dr:state_of_venezuela',
 'Q2977': 'dr:cathedral',
 'Q4057820': 'dr:administrative_okrug_of_moscow',
 'Q140588': 'dr:typhoon',
 'Q183273': 'dr:geographical_pole',
 'Q19364326': 'dr:media_studio_facility',
 'Q35657': 'dr:state_of_the_united_states',
 'Q159675': 'dr:tributary',
 'Q1325302': 'dr:dormant_volcano',
 'Q40080': 'dr:beach',
 'Q55488': 'dr:railway_station',
 'Q188040': 'dr:quarry',
 'Q2354973': 'dr:road_tunnel',
 'Q34876': 'dr:province',
 'Q133056': 'dr:mountain_pass',
 'Q175582': 'dr:egyptian_pyramids',
 'Q23718': 'dr:cardinal_direction',
 'Q502102': 'dr:borehole',
 'Q15310171': 'dr:seaport',
 'Q22911774': 'dr:sierra',
 'Q107679': 'dr:cliff',
 'Q3253281': 'dr:pond',
 'Q4989906': 'dr:monument',
 'Q4232315': 'dr:underground_mine',
 'Q508

In [7]:
arr

[({'coastal_line': 'Q1701967'}, 'coastal_line'),
 ({'typhoon': 'Q140588'}, 'typhoon'),
 ({'geographical_pole': 'Q183273'}, 'geographical_pole'),
 ({'province': 'Q34876'}, 'province'),
 ({'cardinal_direction': 'Q23718'}, 'cardinal_direction'),
 ({'climate_zone': 'Q1639168'}, 'climate_zone'),
 ({'railway': 'Q22667'}, 'railway'),
 ({'tundra': 'Q43262'}, 'tundra'),
 ({'county': 'Q28575'}, 'county'),
 ({'geographical_object': 'Q618123'}, 'geographical_object'),
 ({'watercourse': 'Q355304'}, 'watercourse'),
 ({'province_of_thailand': 'Q50198'}, 'province_of_thailand'),
 ({'quarter': 'Q2983893'}, 'quarter'),
 ({'steppe': 'Q123991'}, 'steppe'),
 ({'continental_shelf': 'Q134851'}, 'continental_shelf'),
 ({'track': 'Q160342'}, 'track'),
 ({'economic_region': 'Q1973901'}, 'economic_region'),
 ({'current_(fluid)': 'Q4456944'}, 'current_(fluid)'),
 ({'flight': 'Q15921555'}, 'flight'),
 ({'latitude': 'Q34027'}, 'latitude'),
 ({'time_zone': 'Q12143'}, 'time_zone'),
 ({'continental_area_and_surroundin

In [111]:
%%time
#Формирование первичных файлов
res = []
for item in tqdm(arr): 
    res.append(dask.delayed(new_file)(item[1] , item[0]))
result = dask.compute(*res)

100%|████████████████████████████████████████████████████████████████████████████████| 72/72 [00:00<00:00, 6542.53it/s]


Wall time: 1.21 s


In [112]:
def filtering_last(filter_file , main_file , ID):
    
    """Функция для проведения фильтрации по Ольгиным файлам"""
    
    res = []
    file_with_filter = f"C:\\Users\\Алексей Кушнер\\sber_basic\\BOCK2\\wikidata\\artificial_classes\\data\\geography\\v8\\clipping\\{filter_file}.xlsx"
    xl = pd.ExcelFile(file_with_filter)
    df = xl.parse(xl.sheet_names[1])
    for k,v in df.iterrows():
        res.append(dask.delayed(entities_sparqul)(v.subclass))
    result = dask.compute(*res)
    result = pd.concat(result).drop_duplicates().reset_index().drop(columns = ['index'])
    result = result[result['classid'].isin(list(result.subclass))]
    result =  result[lambda x : x['subclasslabel'].str[0]!='Q']
    local_lst = []
    for k,v in result.iterrows():
        if (((v.classid in all_id) or (v.classid in local_lst)) and (v.classid != ID)) or (v.subclass in all_id):
            local_lst.append(v.subclass)
            result.drop(k,inplace = True)
    ff = f'excel_with_subclasses/basic/{main_file}.xlsx'
    
    workbook=openpyxl.load_workbook(ff)#Фрагмент для удаления листа и создания нового(приходится вручную ставить на первое место)
    std=workbook.get_sheet_by_name(main_file)
    workbook.remove_sheet(std)
    workbook.save(ff)
    
    new_df = result.groupby(['subclass' , 'subclasslabel']).apply(lambda x: ','.join(list(map(lambda x: '_'.join(x) , zip(x.classid,x.classlabel))))).reset_index()
    new_df.columns = ['subclass' , 'subclasslabel' , 'class']
    with pd.ExcelWriter(f'excel_with_subclasses/basic/{main_file}.xlsx' ,  engine="openpyxl" , mode  = 'a') as writer:
                    new_df.to_excel(writer, sheet_name = main_file , encoding='utf-8-sig' , index = False)

In [113]:
filtering_last("здание" , "building" , "Q41176")
filtering_last("регион" , "region" , "Q82794")
filtering_last("сооружение" , "architectural_structure" , "Q811979")

HTTP Error 429: Too Many RequestsHTTP Error 429: Too Many Requests

HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many RequestsHTTP Error 429: Too Many Requests

HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
Try again  Q17489143 0
Try again  Q488654 0
Try again  Q56195017 0
Try again  Q3044808 0
Try again  Q33316032 0
Try again  Q1552167 0
Try again  Q21550779Try again  0 
Q2001465 0
Try again  Q22908566 0
Try again  Q951781 0
Try again  Q21042623 0
Try again  Q1803690 0
Try again  Q25039918 0
Try again  Q182676 0


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: DeprecationWarning: Call to deprecated function remove_sheet (Use wb.remove(worksheet) or del wb[sheetname]).


HTTP Error 429: Too Many RequestsHTTP Error 429: Too Many RequestsHTTP Error 429: Too Many RequestsHTTP Error 429: Too Many Requests



HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
Try again Try again   Q17016923Q369639Try again Try again   0  0Q57450823
Q2904935  
0Try again Try again 
0  
Q3744088Q4391987  00

Expecting ',' delimiter: line 196675 column 6 (char 5067386)
Try again  Q1496967 0
HTTP Error 429: Too Many RequestsHTTP Error 429: Too Many RequestsHTTP Error 429: Too Many Requests


HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many RequestsHTTP Error 429: Too Many RequestsHTTP Error 429: Too Many RequestsHTTP Error 429: Too Many Requests



HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
Try again  Try again Try again Q199451   Q124282Q525800  0
00

Try again Try again  Try again Q2244608Try again Try again      0Q188669Q17

In [114]:
def entities_sparqul_count(entity_id, f = 0):
    
    """Функция обработчик для файла с количеством представителей"""
    
    res = []
    if f > 5:
        global bad_requests
        bad_requests.append(entity_id)
        print('Bad request ', entity_id)
        return None
    try:
        entities = {}
        raw_results= execute_query(bookquery.subclass_entity_count, entity_id)
        entities['subclass'] = entity_id
        entities['count_P31'] = int(raw_results[0]['entitycount']['value'])
        res.append(entities)
        return pd.DataFrame(data = res, columns = ['subclass', 'count_P31'])
    except Exception as e:
        print(e)
        time.sleep(5)
        print('Try again ', entity_id, f)
        return entities_sparqul_count(entity_id, f + 1)

In [115]:
def creating_file(file):
    
    """Файлы с представителями"""
    
    filename = file.split('\\')[1].split('.')[0]
    if f"{filename}_with_count" in list(map(lambda x: x.split('\\')[1].split('.')[0] , counter_for_files(r"excel_with_subclasses/with_count"))):
        return
    xl = pd.ExcelFile(file)
    for i in range(len(xl.sheet_names)):
        res = []
        df = xl.parse(xl.sheet_names[i])
        for ID in df['subclass']:
            res.append(dask.delayed(entities_sparqul_count)(ID))
        result = dask.compute(*res)
        final_result = pd.concat(result)
        final_result = df.join(final_result.set_index('subclass'), on='subclass').sort_values(
            by = 'count_P31' ,ascending = False)
        if i==0:
            final_result.to_excel(f'excel_with_subclasses/with_count/{filename}_with_count.xlsx', sheet_name = xl.sheet_names[i] ,encoding='utf-8-sig' , index = False)
        else:
            with pd.ExcelWriter(f'excel_with_subclasses/with_count/{filename}_with_count.xlsx' ,  engine="openpyxl" , mode  = 'a') as writer:
                final_result.to_excel(writer, sheet_name = xl.sheet_names[i] , encoding='utf-8-sig' , index = False)
        

In [116]:
%%time
for file in tqdm(counter_for_files(r"excel_with_subclasses/basic")):
    creating_file(file)

  0%|                                                                                           | 0/65 [00:00<?, ?it/s]

HTTP Error 429: Too Many RequestsHTTP Error 429: Too Many RequestsHTTP Error 429: Too Many RequestsHTTP Error 429: Too Many Requests



HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
Try again  Q20180855Try again Try again Try again    Q1738742Q1156810 Q145002960   
000


HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many RequestsHTTP Error 429: Too Many Requests

HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
Try again  Q96578587 0
Try again Try again   Q5447132Q2064236 0 0

Try again  Q7034374 0
HTTP Error 429: Too Many Requests
Try again  Q7708181 0
Try again Try again   Q91325574Q1824509  00

Try again  Q142031 0
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
Try again  Q2144320 0
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many RequestsHTTP Error 429: Too Many Requests

Try again  Q13381944 0
Try agai

HTTP Error 429: Too Many Requests
Try again  Q1067825 0
Try again  Q6972919 0
Try again  Try again Try again Q93431946 Try again    Q81270527Q12241590 0Q1664720  
0
00

Try again  Q1679563 0
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
Try again  Q852811 0
Try again  Q7397337 0
Try again  Q7940971 0
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many RequestsHTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests

HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
Try again Try again   Q414156Q13433904  00

Try again  Q56372086 0
Try again  Q4698554 0
Try again Try again Try again    Q5367899 Q4135597Q47517159 0 
00

Try again  Q3256524 0
Try again Try again Try again    Q965799Q24934356  Q4750816600 
0

HTTP Error 429: Too Man

  2%|█▏                                                                              | 1/65 [03:19<3:32:33, 199.27s/it]

HTTP Error 429: Too Many RequestsHTTP Error 429: Too Many Requests

HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many RequestsHTTP Error 429: Too Many Requests

HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
Try again Try again Try again   Q66361052 Q67702566  Q114100000 
0

Try again  Q55004558 0Try again  
Q41570084 0
Try again  Q27610287 Try again Try again 0 
 Try again  Q4571090Q11435430 Q3284858 00 

0
Try again Try again  Q2312233  Q59053830 
0
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many RequestsHTTP Error 429: Too Many Requests

HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
Try 

...

HTTP Error 429: Too Many Requests
Try again  Q12249052 0
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many RequestsHTTP Error 429: Too Many Requests

HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
Try again  Q3485795 Try again  0Q17166803
 0
Try again  Q2775778 0
Try again Try again   Q12076867Q15210668  00

Try again  Q28659128 0
Try again  Try again Q1454583Try again Try again     Q387917Q901397990Q11088057   
000


Try again Try again  Try again  Q1389310Q49101311  Q1122846 00 

0
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many RequestsHTTP Error 429: Too Many Requests

HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests


 40%|███████████████████████████████▌                                               | 26/65 [05:40<1:07:39, 104.10s/it]

Try again  Try again Try again  Q55774719  Q18330719Try again Try again   Q18652188 00Q59728009Q4875694
  
0 
00

HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
Try again  Q42907160 0
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many RequestsHTTP Error 429: Too Many Requests

HTTP Error 429: Too Many Requests
Try again  Q499720 0
Try again Try again   Q1394653Q95945876  00

Try again  Q11881845 0
Try again  Q131668 0
Try again  Q2679154 0
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
Try again  Q3239205 0
Try again  Q17297280 0
Try again Try again   Q1961599Q20733701  00

Try again  Q62447 0
HTTP Erro

100%|██████████████████████████████████████████████████████████████████████████████████| 65/65 [07:42<00:00,  7.12s/it]


Wall time: 7min 42s


In [125]:
def without_zeros(file):
    
    """Удаление нулей"""
    
    filename = file.split('\\')[1].split('.')[0]
    if f"{filename}_without_zeros" in list(map(lambda x: x.split('\\')[1].split('.')[0] , counter_for_files(r"excel_with_subclasses/without_zeros"))):
        return
    xl = pd.ExcelFile(file)
    for i in range(len(xl.sheet_names)):
        res = []
        df = xl.parse(xl.sheet_names[i])
        df = df[df["count_P31"]>0]
        if len(df)==0:
            continue
        df['signifance'] = ""#Добавляем поле для проверки
        df = df[['subclass' , 'subclasslabel' , 'class' , 'signifance']]
        if i==0:
            df.to_excel(f'excel_with_subclasses/without_zeros/{filename}_without_zeros.xlsx', sheet_name = xl.sheet_names[i] ,encoding='utf-8-sig' , index = False)
        else:
            with pd.ExcelWriter(f'excel_with_subclasses/without_zeros/{filename}_without_zeros.xlsx' ,  engine="openpyxl" , mode  = 'a') as writer:
                df.to_excel(writer, sheet_name = xl.sheet_names[i] , encoding='utf-8-sig' , index = False)
        

In [126]:
%%time
res = []
for file in tqdm(counter_for_files(r"excel_with_subclasses/with_count")):
    res.append(dask.delayed(without_zeros)(file))
result = dask.compute(*res)

100%|███████████████████████████████████████████████████████████████████████████████| 65/65 [00:00<00:00, 13032.02it/s]


Wall time: 1min 8s
